In [1]:
import ntru_config as config

from ntru_utils.NtruEncrypt import *
from ntru_utils.Polynomial import Zx
from ntru_utils.num_to_polynomial import *

import time
import json
from datetime import timedelta
from timeit import default_timer as timer
import pandas as pd

In [2]:
TEST_N = 100

In [3]:
SNPK, SNSK = generate_keypair(config.N_P, config.N_Q, config.N_D, config.N_N)

In [4]:
data = {
    "timestamp": time.time(),
    "pid": {},
}

In [5]:
def concatDataFrame(df, data):
    df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)
    return df


def ntruTrans(message):
    # print(message)
    characterPolynomials, N = koblitz_encoder(
        message, config.N_elliptic_a, config.N_elliptic_b)
    return characterPolynomials, N


def ntruEncrypt(message, publicKey):
    # print(f"{type(message)} -> {message}")
    characterPolynomials, N = ntruTrans(message)
    cipher_polys = []
    for element in characterPolynomials:
        cipher_text = encrypt(element, publicKey, config.N_D, N, config.N_Q)
        cipher_polys.append(cipher_text)
    return cipher_polys, N

In [6]:
df = pd.DataFrame(columns=['Method', 'PID Length', 'Elasped Time (sec)'])

i = 0

while i <= TEST_N:
    t = timer()
    _, _ = ntruEncrypt(json.dumps(data), SNPK)
    # ciper_polys, n = ntruEncrypt(data, SNPK)
    elaspsed_time = timer() - t
    eval_data = {'Method': ['NTRU'], 'PID Length': [f'{i}'],
                 'Elasped Time (sec)': [timedelta(seconds=elaspsed_time).total_seconds()]}

    # print(eval_data)
    df = concatDataFrame(df, eval_data)
    # ====== Next Loop ======
    i += 1
    data['pid'][f'id{i}'] = "AAAA"

print(df.tail())

  Method PID Length Elasped Time (sec)
0   NTRU          0            0.04614
1   NTRU          1           0.042191
2   NTRU          2           0.049376
3   NTRU          3           0.059569
4   NTRU          4           0.069354


In [9]:
print(df.tail())

    Method PID Length Elasped Time (sec)
96    NTRU         96           1.012027
97    NTRU         97           1.093619
98    NTRU         98           1.033953
99    NTRU         99            1.03832
100   NTRU        100            1.06093


In [7]:
# df['Elasped Time (sec)'].mean()

0.5421709702970297

In [8]:
df.to_csv('ntru_n100_server.csv', index=False)